In [1]:
import random
import numpy as np
from read_imageset import read_imageset_arrays
from build_autoencoder import build_autoencoder
from show_original_decoded import show_original_decoded

Using TensorFlow backend.


In [2]:
sz = 128
dataset_name = 'LIDC-IDRI'
x_train = read_imageset_arrays(dataset_name, sz, 0.1)

ValueError: need at least one array to concatenate

In [ ]:
x_test = np.array(random.sample(list(x_train), int(len(x_train)/10)))

In [ ]:
autoencoder, encode_only, decode_only = build_autoencoder(sz, 'adadelta', 'binary_crossentropy')

In [ ]:
autoencoder.fit(x_train, x_train, 
                epochs=250, batch_size=256, 
                shuffle=True, validation_data=(x_test,x_test))

In [ ]:
# save the model(s)
autoencoder.save('models/autoencoder.h5')
encode_only.save('models/encode_only.h5')
decode_only.save('models/decode_only.h5')

In [ ]:
encode_only_imgs = encode_only.predict(x_test)
for n in range(10):
    print("shape of encoded = ", encode_only_imgs[2].shape)
    hist, bins = np.histogram(encode_only_imgs[2])
    print(hist)
    print(bins)

In [ ]:
# add random values to decoded
for n in range(8):
    perturb_vectors = np.zeros(encode_only_imgs[2].shape)
    perturb_element = -120.0 # * np.random.standard_normal()
    perturb_vectors[:,n] = perturb_element
    encode_only_imgs_z = np.add(encode_only_imgs[2], perturb_vectors)

    decoded_imgs = decode_only.predict(encode_only_imgs_z)
    show_original_decoded(x_test, decoded_imgs, sz)

In [ ]:
# add random values to decoded
perturb_vectors = np.random.standard_normal(size=encode_only_imgs[2].shape)
perturb_vectors = np.multiply(perturb_vectors, 15.8)
encode_only_imgs_z = np.add(encode_only_imgs[2], perturb_vectors)

decoded_imgs = decode_only.predict(encode_only_imgs_z)
show_original_decoded(x_test, decoded_imgs, sz)

In [ ]:
random_encoded = 0.0 * np.random.standard_normal(size=encode_only_imgs[2].shape)
print(random_encoded.shape)
print(type(random_encoded))
decoded_imgs_original = decode_only.predict(random_encoded)

perturb_vectors = np.random.standard_normal(size=encode_only_imgs[2].shape)
perturb_vectors = np.multiply(perturb_vectors, 5.0)
decoded_imgs_perturbed = decode_only.predict(np.add(random_encoded, perturb_vectors))

show_original_decoded(decoded_imgs_original, decoded_imgs_perturbed, sz)

In [ ]:
import matplotlib.pyplot as plt
def g(**kwargs):
    plt.figure(2)
    # x = np.linspace(-10,10,num=1000)
    # plt.plot(x,kwargs['1']/100*x + kwargs['2'])
    # plt.ylim(-5,5)
    # plt.show()
    latent = np.array([list(kwargs.values())])
    # print(type(latent))
    # print(latent.shape)
    # latent.reshape(1,)
    # print(latent)
    decoded = decode_only.predict(latent)
    plt.imshow(decoded.reshape(sz,sz))
    plt.gray()
    return kwargs['1']

In [ ]:
from ipywidgets import interactive_output,Layout
import ipywidgets as widgets

In [ ]:
%matplotlib inline
latent_dim = encode_only_imgs[2].shape[-1]
kwargs = {str(k):widgets.IntSlider(value=0,min=-20,max=20,orientation='vertical',layout=Layout(padding='0%')) for k in range(latent_dim)}
w = interactive_output(g,kwargs)
display(widgets.HBox(list(kwargs.values()),layout=Layout(padding='0%')),w)

In [ ]:
encode_only_imgs[2].shape

In [ ]:
!where cudart64_100.dll

In [ ]:
import tensorflow as tf